# EWMA

In [1]:
import numpy as np
import pandas as pd
from numba import jit

In [2]:
@jit(nopython=True)
def ewma(data, alpha, adjust, ignore_na):
    
    old_wt_factor = 1. - alpha
    new_wt = 1. if adjust else alpha
    
    n = data.shape[0]
    output = np.empty(n)  

    weighted_avg = data[0]
    is_observation = (weighted_avg == weighted_avg)
    nobs = int(is_observation)
    output[0] = weighted_avg if (nobs >= 1) else np.nan
    old_wt = 1.
    
    for i in range(1, n):
        cur = data[i]
        is_observation = (cur == cur)
        nobs += int(is_observation)
        if weighted_avg == weighted_avg:

            if is_observation or (not ignore_na):

                old_wt *= old_wt_factor
                if is_observation:

                    if weighted_avg != cur:
                        weighted_avg = ((old_wt * weighted_avg) +
                                        (new_wt * cur)) / (old_wt + new_wt)
                    if adjust:
                        old_wt += new_wt
                    else:
                        old_wt = 1.
        elif is_observation:
            weighted_avg = cur

        output[i] = weighted_avg if (nobs >= 1) else np.nan

    return output

In [3]:
data = np.arange(1e5) * 1.0
data[3] = np.nan
data[4] = np.nan

In [4]:
expected = pd.Series(data).ewm(alpha=0.1, adjust=True, ignore_na=False).mean().values

In [5]:
output = ewma(data, alpha=0.1, adjust=True, ignore_na=False)

In [6]:
np.allclose(expected, output)

True

In [7]:
%timeit pd.Series(data).ewm(alpha=0.1, adjust=True, ignore_na=False).mean().values

100 loops, best of 3: 9.16 ms per loop


In [8]:
%timeit ewma(data, alpha=0.1, adjust=True, ignore_na=False)

1000 loops, best of 3: 726 µs per loop
